# 言語処理100本ノック 2020 第４章

Reference: https://nlp100.github.io/ja/ch04.html

In [1]:
! wget https://nlp100.github.io/data/neko.txt

--2020-04-13 06:49:05--  https://nlp100.github.io/data/neko.txt
Resolving nlp100.github.io...185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to nlp100.github.io|185.199.108.153|:443...connected.
OpenSSL: error:1407742E:SSL routines:SSL23_GET_SERVER_HELLO:tlsv1 alert protocol version
Unable to establish SSL connection.
